In [26]:
import pandas as pd
import numpy as np

In [60]:
df = pd.read_csv('merged_df.csv')

C:\Users\chcro\AppData\Local\Temp\ipykernel_8200\3228167239.py:1: DtypeWarning: Columns (29,70,71,81,84,85,86,87,88) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('merged_df.csv')


In [61]:
# replace all white spaces in column names with _
df.columns = df.columns.str.replace(' ', '_')

# replace \n in column names with _
df.columns = df.columns.str.replace('\n', '_')
df.columns = df.columns.str.replace('__', '_')

# more cleanup and data preprocessing
#final_df = df.dropna()
#df = df.drop_duplicates()
df = df.reset_index(drop=True)

# convert dates to datetime and sort by date
df['DATE'] = pd.to_datetime(df['DATE'])
df['R'] = df['OR'] + df['DR']

In [65]:
def assign_results(group):
    """
    Assigns spread results to each team in a game based on their final scores and closing spreads.

    Parameters:
    - group (DataFrame): A DataFrame containing data for a single game, grouped by 'GAME-ID'.

    Returns:
    - DataFrame: The original group DataFrame with an added 'spread_result' column.
    """
    teams = group['TEAM'].unique()
    
    # Ensure that there are exactly two teams in the game
    if len(teams) != 2:
        # If not exactly two teams, assign 0 to all (or handle as needed)
        group['spread_result'] = 0
        return group
    
    # Extract team names
    team1, team2 = teams[0], teams[1]
    
    # Extract the rows corresponding to each team
    t1_row = group[group['TEAM'] == team1].iloc[0]
    t2_row = group[group['TEAM'] == team2].iloc[0]
    
    # Calculate the spread factors
    t1_spread_f = t1_row['F'] + t1_row['CLOSING_SPREAD']
    t2_spread_f = t2_row['F'] + t2_row['CLOSING_SPREAD']
    
    # Determine which team has the lower spread
    # (This part of the logic is unclear in your original code; adjust as needed)
    # For demonstration, assuming you want to determine which team's spread covered the opponent's score
    res = [
        t1_spread_f > t2_row['F'],  # Team1 covers Team2's score
        t2_spread_f > t1_row['F']   # Team2 covers Team1's score
    ]
    
    # Initialize 'spread_result' to 0
    group['spread_result'] = 0
    
    # Assign spread results using .loc to avoid SettingWithCopyWarning
    group.loc[group['TEAM'] == team1, 'spread_result'] = int(res[0])
    group.loc[group['TEAM'] == team2, 'spread_result'] = int(res[1])
    
    return group

# Example usage:
# Assuming 'df' is your DataFrame with columns ['GAME-ID', 'TEAM', 'F', 'CLOSING_SPREAD', ...]
# Apply the function to each group
df = df.groupby('GAME-ID').apply(assign_results).reset_index(drop=True)

# Verify that 'spread_result' column exists
print(df.head())


C:\Users\chcro\AppData\Local\Temp\ipykernel_8200\2807352888.py:50: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('GAME-ID').apply(assign_results).reset_index(drop=True)


            BIGDATABALL_DATASET   GAME-ID       DATE  PLAYER-ID  \
0  NBA 2018-2019 Regular Season  21800001 2018-10-16     203967   
1  NBA 2018-2019 Regular Season  21800001 2018-10-16     203496   
2  NBA 2018-2019 Regular Season  21800001 2018-10-16     203954   
3  NBA 2018-2019 Regular Season  21800001 2018-10-16    1628365   
4  NBA 2018-2019 Regular Season  21800001 2018-10-16    1627732   

   PLAYER_FULL_NAME POSITION      OWN_TEAM OPPONENT_TEAM VENUE_(R/H)  \
0       Dario Saric        F  Philadelphia        Boston           R   
1  Robert Covington        F  Philadelphia        Boston           R   
2       Joel Embiid      C-F  Philadelphia        Boston           R   
3    Markelle Fultz        G  Philadelphia        Boston           R   
4       Ben Simmons      F-G  Philadelphia        Boston           R   

  STARTER_(Y/N)  ...  CLOSING_TOTAL  MONEYLINE  HALFTIME  BOX_SCORE_URL  \
0             Y  ...          211.5        170       106            NaN   
1             

In [67]:
df.sort_values(by='DATE').tail()

,BIGDATABALL_DATASET,GAME-ID,DATE,PLAYER-ID,PLAYER_FULL_NAME,POSITION,OWN_TEAM,OPPONENT_TEAM,VENUE_(R/H),STARTER_(Y/N),...,CLOSING_TOTAL,MONEYLINE,HALFTIME,BOX_SCORE_URL,ODDS_URL,BIGDATABALL_DATASET,FULL_GAME_ODDS_URL,CREW_CHIEF,REFEREE_&_UMPIRE,spread_result
160450,NBA 2024 Playoffs,42300405,2024-06-17,1629655,Daniel Gafford,C-F,Dallas,Boston,R,Y,...,210.5,215,102o 15,NaN,NaN,NBA 2024 Playoffs,NaN,Bill Kennedy,Zach Zarba,0
160449,NBA 2024 Playoffs,42300405,2024-06-17,1629023,P.J. Washington,F,Dallas,Boston,R,Y,...,210.5,215,102o 15,NaN,NaN,NBA 2024 Playoffs,NaN,Bill Kennedy,Zach Zarba,0
160448,NBA 2024 Playoffs,42300405,2024-06-17,1627884,Derrick Jones Jr.,F,Dallas,Boston,R,Y,...,210.5,215,102o 15,NaN,NaN,NBA 2024 Playoffs,NaN,Bill Kennedy,Zach Zarba,0
160461,NBA 2024 Playoffs,42300405,2024-06-17,1641765,Olivier-Maxence Prosper,F,Dallas,Boston,R,N,...,210.5,215,102o 15,NaN,NaN,NBA 2024 Playoffs,NaN,Bill Kennedy,Zach Zarba,0
160454,NBA 2024 Playoffs,42300405,2024-06-17,1630182,Josh Green,G,Dallas,Boston,R,N,...,210.5,215,102o 15,NaN,NaN,NBA 2024 Playoffs,NaN,Bill Kennedy,Zach Zarba,0


In [68]:
df.groupby(['GAME-ID', 'TEAM'])[['3P', '3PA', 'OR', 'DR', 'TO', 'FT', 'spread_result']].sum()

3P  3PA  OR  DR  TO  FT  spread_result
GAME-ID  TEAM                                                 
21800001 Boston         11   37  12  43  14  10             13
         Philadelphia    5   26   6  41  16  14              0
21800002 Golden State    7   26  17  41  21  17              0
         Oklahoma City  10   37  16  29  14  24             10
21800003 Charlotte      16   38   9  32  11  14             10
...                     ..  ...  ..  ..  ..  ..            ...
52300201 Miami          14   33   9  38  15  22             12
52300211 New Orleans     7   19  11  34  15  10              9
         Sacramento     11   41  14  26  15  15              0
62300001 Indiana        10   41   9  23   9  29              0
         LA Lakers       2   13  12  43  18  27             12

[15176 rows x 7 columns]

In [73]:
df.groupby(['GAME-ID', 'OPPONENT_TEAM', 'OWN_TEAM', 'POSITION'])['PTS'].sum().reset_index().tail(20)

,GAME-ID,OPPONENT_TEAM,OWN_TEAM,POSITION,PTS
62981,52300201,Miami,Chicago,C,18
62982,52300201,Miami,Chicago,F,8
62983,52300201,Miami,Chicago,F-G,22
62984,52300201,Miami,Chicago,G,43
62985,52300211,New Orleans,Sacramento,C,0
62986,52300211,New Orleans,Sacramento,C-F,23
62987,52300211,New Orleans,Sacramento,F,30
62988,52300211,New Orleans,Sacramento,G,45
62989,52300211,Sacramento,New Orleans,C,19
62990,52300211,Sacramento,New Orleans,C-F,13


In [74]:
cols = ['BIGDATABALL_DATASET', 'GAME-ID', 'DATE', 'PLAYER-ID',
       'PLAYER_FULL_NAME', 'POSITION', 'OWN_TEAM', 'OPPONENT_TEAM',
       'VENUE_(R/H)', 'STARTER_(Y/N)', 'MIN', 'FG', 'FGA', '3P', '3PA', 'FT',
       'FTA', 'OR', 'DR', 'TOT', 'A', 'PF', 'ST', 'TO', 'BL', 'PTS',
       'USAGE_RATE_(%)', 'DAYS_REST', 'R', 'DATASET', 'DATE_team', 'TEAM',
       'VENUE', '1Q', '2Q', '3Q', '4Q', 'OT1', 'OT2', 'OT3', 'OT4', 'OT5', 'F',
       'MIN_team', 'FG_team', 'FGA_team', '3P_team', '3PA_team', 'FT_team',
       'FTA_team', 'OR_team', 'DR_team', 'TOT_team', 'A_team', 'PF_team',
       'ST_team', 'TO_team', 'TO_TO', 'BL_team', 'PTS_team', 'POSS', 'PACE',
       'OEFF', 'DEFF', 'TEAM_REST_DAYS', 'MAIN_REF', 'CREW',
       'OPENING_ODDS', 'OPENING_SPREAD', 'OPENING_TOTAL', 'LINE_MOVEMENT_#1',
       'LINE_MOVEMENT_#2', 'LINE_MOVEMENT_#3', 'CLOSING_ODDS',
       'CLOSING_SPREAD', 'CLOSING_TOTAL', 'MONEYLINE' 'CREW_CHIEF', 'REFEREE_&_UMPIRE',
       'spread_result']

Index(['BIGDATABALL_DATASET', 'GAME-ID', 'DATE', 'PLAYER-ID',
       'PLAYER_FULL_NAME', 'POSITION', 'OWN_TEAM', 'OPPONENT_TEAM',
       'VENUE_(R/H)', 'STARTER_(Y/N)', 'MIN', 'FG', 'FGA', '3P', '3PA', 'FT',
       'FTA', 'OR', 'DR', 'TOT', 'A', 'PF', 'ST', 'TO', 'BL', 'PTS',
       'USAGE_RATE_(%)', 'DAYS_REST', 'R', 'DATASET', 'DATE_team', 'TEAM',
       'VENUE', '1Q', '2Q', '3Q', '4Q', 'OT1', 'OT2', 'OT3', 'OT4', 'OT5', 'F',
       'MIN_team', 'FG_team', 'FGA_team', '3P_team', '3PA_team', 'FT_team',
       'FTA_team', 'OR_team', 'DR_team', 'TOT_team', 'A_team', 'PF_team',
       'ST_team', 'TO_team', 'TO_TO', 'BL_team', 'PTS_team', 'POSS', 'PACE',
       'OEFF', 'DEFF', 'TEAM_REST_DAYS', 'STARTING_LINEUPS', 'Unnamed:_38',
       'Unnamed:_39', 'Unnamed:_40', 'Unnamed:_41', 'MAIN_REF', 'CREW',
       'OPENING_ODDS', 'OPENING_SPREAD', 'OPENING_TOTAL', 'LINE_MOVEMENT_#1',
       'LINE_MOVEMENT_#2', 'LINE_MOVEMENT_#3', 'CLOSING_ODDS',
       'CLOSING_SPREAD', 'CLOSING_TOTAL', 'MONEYLINE